In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
import skfuzzy as fuzz

In [2]:
# 1. Cargar los datasets
pathSmall = '../Datasets/ml-latest-small/'
ratings = pd.read_csv(pathSmall + 'ratings.csv')  # Debe tener columnas: userId, movieId, rating, timestamp
ratings.drop(columns=['timestamp'], inplace=True)  # Eliminar la columna timestamp

In [3]:
# === 2. Crear matriz de utilidad ===
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
user_ids = user_item_matrix.index
movie_ids = user_item_matrix.columns

In [ ]:
# === 3. Fuzzy C-Means ===
# Necesitamos transponer para que cada usuario sea una columna como lo requiere skfuzzy
data = user_item_matrix.values.T  # [n_items x n_users]
n_clusters = 5

# Ejecutar FCM
centers, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data, c=n_clusters, m=1.7, error=0.5, maxiter=1000, init=None
)
u = u.T  # [n_users x n_clusters]

print(u)

[[0.19946302 0.19841808 0.20057842 0.2010916  0.20044888]
 [0.20293599 0.20723244 0.19625599 0.19475913 0.19881645]
 [0.20374913 0.20870268 0.19491877 0.19417999 0.19844942]
 ...
 [0.19899743 0.19761926 0.20131248 0.20159476 0.20047607]
 [0.20418497 0.20985741 0.19480395 0.19422972 0.19692394]
 [0.19938521 0.19867071 0.20085485 0.20078986 0.20029936]]


In [7]:
def defuzz_cog(u):
    """
    Calcula centro de gravedad de las membresías y redondea al cluster más cercano.
    Los índices de cluster se asumen 1..c.
    Output es array de etiquetas 0..c-1.
    """
    n_clusters, n_samples = u.shape
    # índices 1..c
    indices = np.arange(1, n_clusters + 1)
    # center of gravity: sum(mu_i * i) / sum(mu_i)
    cog = (u.T @ indices) / u.sum(axis=0)
    # redondear y mapear de vuelta a [0..c-1]
    labels = np.rint(cog).astype(int) - 1
    # asegurar límites
    labels = np.clip(labels, 0, n_clusters - 1)
    return labels

labels = defuzz_cog(u.T)
print(labels)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 